In [2]:
import os
import pathlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
# delete bad images

data_path = '/aiffel/aiffel/CustomM/model-fit/data/30vnfoods/'
train_path = data_path + 'Train/'
test_path = data_path + 'Test/'

for path in [train_path, test_path]:
    classes = os.listdir(path)
    
    for food in classes:
        food_path = os.path.join(path, food)
        images = os.listdir(food_path)
        
        for image in images:
            with open(os.path.join(food_path, image), 'rb') as f:
                bytes = f.read()
            if bytes[:3] != b'\xff\xd8\xff':
                print(os.path.join(food_path, image))
                os.remove(os.path.join(food_path, image))

In [4]:
classes = os.listdir(train_path)
train_len = 0

for food in classes:
    food_path = os.path.join(train_path, food)
    images = os.listdir(food_path)
    
    train_len += len(images)
    
str(train_len)
classes

['Banh mi',
 'Pho',
 'Banh cuon',
 'Banh xeo',
 'Bun bo Hue',
 'Bun dau mam tom',
 'Chao long',
 'Banh khot',
 'Com tam',
 'Bun rieu']

In [5]:
# dataloader

class_names = ['Banh mi', 'Pho', 'Banh cuon', 'Banh xeo', 'Bun bo Hue']
               


def process_path(file_path, class_names=class_names, img_shape=(224, 224)):
    
    label = tf.strings.split(file_path, os.path.sep)[-2]
    label = tf.argmax(label == class_names)
    
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, img_shape)
    img = img / 255.0
    
    return img, label



def prepare_for_training(ds, epochs=5, batch_size=32, cache=True, shuffle_buffer_size=100):

    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()
    ds = ds.shuffle(buffer_size = shuffle_buffer_size)
    ds = ds.repeat(epochs)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size = tf.data.AUTOTUNE)
    
    return ds




def load_data(data_path, class_names, batch_size=32):
    all_paths = []
    for class_name in class_names:
        class_path = os.path.join(data_path, class_name)
        class_files = [os.path.join(class_path, fname) for fname in os.listdir(class_path)]
        all_paths.extend(class_files)

    # Shuffle the list to mix up class distributions
    np.random.shuffle(all_paths)

    list_ds = tf.data.Dataset.from_tensor_slices(all_paths)
    list_ds = list_ds.map(lambda x: process_path(x, class_names),
                          num_parallel_calls=tf.data.AUTOTUNE)
    return prepare_for_training(list_ds, batch_size=batch_size)



In [6]:
# create model

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers

class Model(tf.keras.Model):
    
    
    def __init__(self, num_classes=10, freeze=False):
        super(Model, self).__init__()
        self.base_model = EfficientNetB0(include_top=False,
                                        input_shape=(224, 224, 3), weights='imagenet')
        
        if freeze:
            for layer in self.base_model.layers:
                layer.trainable = False
                
        self.global_avg_layer = layers.GlobalAveragePooling2D()
        self.output_layer = layers.Dense(num_classes, activation='softmax')
    
        
        
    def call(self, inputs, training=True):
        x = self.base_model(inputs, training=training)
        x = self.global_avg_layer(x)
        return self.output_layer(x)
        

In [9]:
# custom trainer

from tqdm import tqdm
from tqdm.notebook import tqdm


class Trainer:
    def __init__(self, model, epochs, batch, loss_fn, optimizer):
        self.model = model
        self.epochs = epochs
        self.batch = batch
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        
    def train(self, train_dataset, train_metric):
            
        progbar = tqdm(total=len(train_dataset) * self.epochs, miniters=1, mininterval=0.1)

            
        for epoch in range(self.epochs):
            print('\nStart of epoch %d' % (epoch +1,))
            
            
            for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
                with tf.GradientTape() as tape:
                    logits = self.model(x_batch_train, training=True)
                    loss_value = self.loss_fn(y_batch_train, logits)
                grads = tape.gradient(loss_value, self.model.trainable_weights)
                self.optimizer.apply_gradients(zip(grads, self.model.trainable_weights))
            
                train_acc_metric.update_state(y_batch_train, logits)
                progbar.update(1)
                progbar.set_postfix(loss=loss_value.numpy(), epoch=epoch+1)

            train_acc = train_acc_metric.result()
            print('training acc over epoch: %.4f' % (float(train_acc),))
            train_acc_metric.reset_states()
            


In [10]:
# now train!

train_path = "/aiffel/aiffel/CustomM/model-fit/data/30vnfoods/Train"

epoch = 3
batch = 16



model = Model(num_classes=10)
dataset = load_data(data_path=train_path, class_names=class_names, batch_size=batch)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
trainer = Trainer(model=model,
                 epochs=epoch,
                 batch=batch,
                 
                 loss_fn=loss_fn,
                 optimizer=optimizer)

trainer.train(train_dataset=dataset, 
             train_metric=train_acc_metric)



  0%|          | 0/5031 [00:00<?, ?it/s]


Start of epoch 1
training acc over epoch: 0.9132

Start of epoch 2
training acc over epoch: 0.9676

Start of epoch 3
training acc over epoch: 0.9774


In [13]:
# test model

test_ds = load_data(data_path=test_path, class_names=class_names)

for step_train, (x_batch_train, y_batch_train) in enumerate(test_ds.take(10)):
    prediction = model(x_batch_train)
    
    correct_predictions = tf.equal(y_batch_train, tf.argmax(prediction, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
    
    print("{}/{}".format(tf.reduce_sum(tf.cast(correct_predictions, tf.int32)).numpy(),
                         y_batch_train.shape[0]))


31/32
28/32
27/32
29/32
31/32
26/32
26/32
30/32
28/32
28/32


# 회고
- 정확도는 80-90 후반대로 높게 나왔다
- 사실 코드를 정확하게 이해하고 직접 짜지 못해서 GPT 의 하드캐리를 받았는데, 여기저기 문제가 생겨 계속 에러가 났다
- 트레이닝 시간이 너무 오래걸려서 결국 데이터셋/클래스를 반으로 뚝 자르고 에포크도 3개만 돌렸다
- GPT 가 간단히 해결해 준 것들도 있고, 전혀 딴소리만 해서 내가 이건가? 하고 때려맞췄는데 맞았던 것도 있고, 계속 답변이 바뀌어서 클로드랑 경쟁시켰던 것도 있는데, 결론적으로 잘 작동해서 기쁘다!
- 이런 게 문제 해결의 즐거움일까? 지피티가 어서 더 똑똑해졌으면!